# 开头和结尾都是两个下划线为魔术方法
> 这样的拼写表示名称有特殊意义，因此绝不要在程序中创建这样的名称。在这样的名称中，很大一部分都是魔法（特殊）方法的名称。如果你的对象实现了这些方法，它们将在特定情况下（具体是哪种情况取决于方法的名称）被Python调用，而几乎不需要直接调用。

In [4]:
# 默认构造函数
class FooBar:
    def __init__(self):# __int__为构造函数 __del__为析构  
        self.somevar = 42
f = FooBar() # 自动调用__init__构造函数
f.somevar

42

In [10]:
# 在继承中使用构造函数 注意要调用基类的构造函数
class Bird:
    def __init__(self):
        self.hungry  = True
    def eat(self):
        if self.hungry:
            print('Aaaah ...')   
            self.hungry = False 
        else: 
            print('No, thanks!')


class SongBird(Bird):
    def __init__(self):   
        # 旧版 Bird.__init__(self)  # 主要需要初始化基类  
        super().__init__() # 好处 多个基类也只需一行 无需理会复杂的派生关系
        self.sound = 'Squawk!' 
    def sing(self): 
        print(self.sound)
sb = SongBird()
sb.sing()
sb.eat()

Squawk!
Aaaah ...


# 迭代器
# 实现了方法`__iter__`的对象是可迭代的，而实现了方法`__next__`的对象是迭代器

In [14]:
class Fibs:
    def __init__(self):  
        self.a = 0     
        self.b = 1 
    def __next__(self):
        if self.b > 30: # 超过30停止迭代器
            raise StopIteration
        self.a, self.b = self.b, self.a + self.b 
        return self.a 
    def __iter__(self): 
        return self


fibs = Fibs() # 创建了迭代器
for i in fibs:
    print(i)


1
1
2
3
5
8
13
21


# 生成器是一种使用普通函数语法定义的迭代器 是很新的概念

In [21]:
# 创建一个遍历以下列表的函数迭代器
nested = [[1, 2], [3, 4], [5]]
def flatten(nested): 
    for sublist in nested: 
        for element in sublist: 
            yield element
print(*flatten(nested))
for i in flatten(nested):
    print(i,end=' ')

# 生成器推导 
# 其工作原理与列表推导相同，但不是创建一个列表（即不立即执行循环），而是返回一个生成器，让你能够逐步执行计算
g = ((i + 2) ** 2 for i in range(2, 27))
print('\n',*g)
#直接在一对既有的圆括号内（如在函数调用中）使用生成器推导时，无需再添加一对圆括号
sum(i ** 2 for i in range(10))


1 2 3 4 5
1 2 3 4 5 
 16 25 36 49 64 81 100 121 144 169 196 225 256 289 324 361 400 441 484 529 576 625 676 729 784


285

In [22]:
# 如果给定无穷嵌套呢 可以使用递归式生成器
def flatten(nested): 
    try:
        # 不迭代类似于字符串的对象： 
        try: nested + ''  
        except TypeError: 
            pass  
        else: raise TypeError
        for sublist in nested: 
            for element in flatten(sublist): 
                yield element 
    # 当nested为一个数 for循环将引发TypeError异常（因为你试图迭代一个数），而这个生成器只生成一个元素
    except TypeError:  
        yield nested


list(flatten([[[1], 2], 3, 4, [5, [6, 7]], 8]))


[1, 2, 3, 4, 5, 6, 7, 8]

# 实战应用:8皇后

In [23]:
# state[i]表示第i个皇后处于的位置

# 检测下一个皇后的位置为nextX时 是否和之前的冲突
def conflict(state,nextX):
    nextY = len(state) # 当前皇后的数量
    for i in range(nextY):
        if abs(state[i]-nextX) in (0,nextY-i):#要保证不在同一列或对角线上
            return True
    return False

In [31]:
def queens(num=8, state=()): 
    for pos in range(num): 
        if not conflict(state, pos): 
            if len(state) == num-1: 
                yield (pos,)  # 记得(pos,)中的逗号必不可少（不能仅用圆括号将pos括起），这样得到的才是元组
            else: 
                for result in queens(num, state + (pos,)): 
                    yield (pos,) + result


def prettyprint(solution): 
    def line(pos, length=len(solution)): 
        return '. ' * (pos) + 'X ' + '. ' * (length-pos-1) 
    for pos in solution: 
        print(line(pos))

import random
prettyprint(random.choice(list(queens(8)))) # 随机选取一个解 打印出来


. . . X . . . . 
. . . . . . X . 
. . . . X . . . 
. X . . . . . . 
. . . . . X . . 
X . . . . . . . 
. . X . . . . . 
. . . . . . . X 
